<a href="https://colab.research.google.com/github/bkvkrll/-Introduction-to-Natural-Language-Processing/blob/main/lesson7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Урок 7. Сверточные нейронные сети для анализа текста
Задание
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек
2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [2]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=2dc1c7a42cbb5828e53270d1ba9466c6f556dbf6fcc164a8d1dc19f63354be65
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [3]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 8.2 MB 10.0 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=f8638e703d75358930a47b25b929c96ba226b6f6717ac2d701181b1925b436e4
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [4]:
!pip install navec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from spacy.lang.ru.stop_words import STOP_WORDS
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from navec import Navec

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical, get_file
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import Constant

In [6]:
!pip install xlrd==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 8.3 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 2.0.1
    Uninstalling xlrd-2.0.1:
      Successfully uninstalled xlrd-2.0.1


In [6]:
MAX_WORDS = 2000
MAX_LEN = 100

In [7]:
data = pd.read_excel('/content/отзывы за лето (1).xls')
le = LabelEncoder()
data['Rating'] = le.fit_transform(data['Rating'])
data.sample(5)

,Rating,Content,Date
20087,0,Не скачать вообще приложение. Постоянно ошибку...,2017-06-15
14616,4,Ок,2017-07-23
17952,3,Xiaomi redmi 4 pro не сохраняет настройку вход...,2017-07-08
20565,4,Лол кек чебурек,2017-06-03
9093,4,Все нравится,2017-08-01


In [8]:
sw = set((get_stop_words("ru") + list(STOP_WORDS))) - {'не', 'ни', 'нет'}
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt, exclude=exclude, sw=sw, morpher=morpher):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
#     txt = re.sub("не\s", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [9]:
data['preprocessed_text'] = data['Content'].apply(preprocess_text)

In [10]:
data.sample(5)

,Rating,Content,Date,preprocessed_text
1295,4,Грузится долго,2017-08-12,грузиться
2757,2,Ужас,2017-08-10,ужас
9036,3,Гуд,2017-08-01,гуд
14438,4,Очень удобное приложение. Пользуюсь постоянно.,2017-07-23,удобный приложение пользоваться постоянно
7531,4,"Все отлично,иногда что то не получается ,но ск...",2017-08-03,отличноиногда не получаться причина интернетап...


In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
train_corpus = " ".join(data["preprocessed_text"])
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]
len(tokens_filtered)

102261

In [13]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(MAX_WORDS-1)]
tokens_filtered_top[:7]

['приложение', 'не', 'удобно', 'работать', 'удобный', 'отлично', 'нравиться']

In [14]:
vocabulary = {v: k for k, v in enumerate(tokens_filtered_top, 1)}

In [15]:
len(vocabulary)

1999

In [16]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [17]:
data_train = np.asarray([text_to_sequence(text, MAX_LEN) for text in data['preprocessed_text']], dtype=np.int32)

In [18]:

X_train, X_test, y_train, y_test = train_test_split(data_train, data['Rating'], test_size=0.33)

Cлой Embedding

In [19]:
BATCH_SIZE = 1024
EPOCHES = 10

In [20]:
model = Sequential()
model.add(Embedding(input_dim=MAX_WORDS, output_dim=128, input_length=MAX_LEN))
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(20))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(Dense(5))
model.add(Activation('softmax'))

In [21]:
opt = Adam(learning_rate=0.001)
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer=opt,
              metrics=['accuracy'])

In [22]:

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          256000    
                                                                 
 conv1d (Conv1D)             (None, 96, 128)           82048     
                                                                 
 activation (Activation)     (None, 96, 128)           0         
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 20)                2580      
                                                                 
 activation_1 (Activation)   (None, 20)                0         
                                                        

In [23]:
history = model.fit(X_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHES,
                    verbose=0,
                    validation_split=0.1)

In [24]:
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 1s 194ms/step - loss: 0.6393 - accuracy: 0.7774


Test loss: 0.6393414735794067
Test accuracy: 0.7773540616035461


Embedding с предобученными векторами

In [25]:
path = '/content/navec_news_v1_1B_250K_300d_100q.tar'
emb = Navec.load(path)

In [26]:
embedding_dim = 300

embedding_matrix = np.zeros((MAX_WORDS, embedding_dim))
for word in tokens_filtered_top:
    embedding_vector = emb.get(word)
    if embedding_vector is not None:
        embedding_matrix[tokens_filtered_top.index(word)] = embedding_vector

In [27]:
embedding_matrix.shape

(2000, 300)

In [28]:
embedding_layer = Embedding(
    MAX_WORDS,
    embedding_dim,
    embeddings_initializer=Constant(embedding_matrix),
    trainable=False,
)

In [29]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(20))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(Dense(5))
model.add(Activation('softmax'))

In [30]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [31]:
history = model.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHES,
          verbose=0,
          validation_split=0.1)

In [32]:
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 3s 438ms/step - loss: 0.6889 - accuracy: 0.7652


Test loss: 0.6888515949249268
Test accuracy: 0.7651804089546204


Существенной разницы с предобученными векторами не получил